## 3.3 通过自注意力机制关注输入的不同部分

![from comp5046](imgs/PixPin_2025-07-06_04-33-57.png)

**Self-Attention**: A weighted average of vectors where the weights and input vectors are all from the input

![Alt text](imgs/PixPin_2025-07-06_04-36-28.png)

图 3.7 显示了一个输入序列，记作 x，由 T 个元素组成，表示为 x(1) 到 x(T)。该序列通常代表文本，例如一个句子，并且该文本已被转换为 token 嵌入。

举例来说，假设输入文本为 “Your journey starts with one step”。在这个例子中，序列中的每个元素（如 x(1)）对应一个 d 维的嵌入向量，用于表示特定的 token，例如 “Your”。在图 3.7 中，这些输入向量显示为 3 维的嵌入向量。

在自注意力机制中，我们的**目标**是为输入序列中的每个元素 x(i) 计算其对应的上下文向量 z(i) 。为了说明这个概念，我们聚焦于第二个输入元素 x(2) 的嵌入向量（对应于词 "journey"）以及相应的上下文向量 z(2)，如图 3.7 底部所示。这个增强的上下文向量 z(2) 也是一个嵌入向量，包含了关于 x(2) 以及序列中所有其他输入元素 x(1) 到 x(T) 的语义信息。

In [1]:
import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)


![Alt text](imgs/PixPin_2025-07-06_04-40-41.png)

这里qi是Gelato，kj可以是is或者delicious。但还有问题，首先weighted是线性的，并且这里并没有词序的关系,因此我们引入了之前提到的位置编码。

![Alt text](imgs/PixPin_2025-07-06_04-41-22.png)

![Alt text](imgs/PixPin_2025-07-06_04-42-44.png)

实现自注意力机制的第一步是计算中间值 ω，即注意力得分，如图 3.8 所示。（请注意，图 3.8 中展示的输入张量值是截断版的，例如，由于空间限制，0.87 被截断为 0.8。在此截断版中，单词 "journey" 和 "starts" 的嵌入向量可能会由于随机因素而看起来相似）。

![Alt text](imgs/PixPin_2025-07-06_04-43-28.png)


根据input，我们可以设计一个小demo：
首先，我们要制定Q，K，V的权重矩阵。

In [2]:
W_Q = torch.tensor([[0.1, 0.3, 0.5],  # 3x3
                    [0.2, 0.4, 0.6],
                    [0.3, 0.5, 0.7]])

W_K = torch.tensor([[0.2, 0.1, 0.4],
                    [0.3, 0.2, 0.5],
                    [0.4, 0.3, 0.6]])

W_V = torch.tensor([[0.5, 0.4, 0.3],
                    [0.6, 0.5, 0.2],
                    [0.7, 0.6, 0.1]])


然后，计算绝对位置编码。

In [3]:
import torch.nn.functional as F

def positional_encoding(pos, d_model):
    pe = torch.zeros(d_model)
    for i in range(0, d_model, 2):
        angle = torch.tensor(pos / (10000 ** ((2 * i)/d_model)))
        pe[i] = torch.sin(angle)

        if i + 1 < d_model:
            angle = torch.tensor(pos / (10000 ** ((2 * i)/d_model)))
            pe[i+1] = torch.cos(angle)
    return pe
    
pos_encodings = torch.stack([positional_encoding(pos, inputs.shape[1]) for pos in range(1, 7)])

inputs_with_pe = inputs + pos_encodings

In [4]:
# 计算 Q, K, V
Q = inputs_with_pe @ W_Q  # (6,3)
K = inputs_with_pe @ W_K  # (6,3)
V = inputs_with_pe @ W_V  # (6,3)

# 计算 attention scores (Q @ K.T / sqrt(d_k))
d_k = Q.shape[1]
scores = Q @ K.T / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))

# Softmax 归一化
attention_weights = F.softmax(scores, dim=-1)

# 加权求和
output = attention_weights @ V

import pandas as pd

# 拼接输入、注意力权重、输出结果
df_inputs = pd.DataFrame(inputs_with_pe.numpy(), columns=['x1', 'x2', 'x3'])
df_attention = pd.DataFrame(attention_weights.numpy(), columns=[f"T{i+1}" for i in range(6)])
df_output = pd.DataFrame(output.numpy(), columns=['o1', 'o2', 'o3'])

# 合并所有 DataFrame
result_df = pd.concat([df_inputs, df_attention, df_output], axis=1)

# 打印结果
print("\n===== Self-Attention 结果 (含位置编码) =====\n")
print(result_df.to_string(index=False))



===== Self-Attention 结果 (含位置编码) =====

       x1        x2       x3       T1       T2       T3       T4       T5       T6       o1       o2       o3
 1.271471  0.690302 0.890005 0.315381 0.239435 0.105702 0.044346 0.066093 0.229044 1.273928 1.060221 0.435727
 1.459297  0.453853 0.660009 0.296198 0.233471 0.115046 0.054239 0.076627 0.224420 1.236005 1.028942 0.420495
 0.711120 -0.139992 0.640014 0.231497 0.205499 0.144342 0.099181 0.117838 0.201644 1.086208 0.905413 0.360154
-0.536803 -0.073644 0.330019 0.162206 0.163468 0.167715 0.172386 0.170190 0.164035 0.885203 0.739702 0.278801
-0.188924  0.533662 0.100023 0.193902 0.184244 0.158497 0.135087 0.145366 0.182904 0.982739 0.820102 0.318358
-0.229415  1.760170 0.550028 0.288950 0.230573 0.118408 0.058350 0.080746 0.222973 1.221021 1.016597 0.414371


```python
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
```

```x1, x2, x3```是原始词向量 + 绝对位置编码，即 inputs_with_pe

```xml
原始向量: [0.43, 0.15, 0.89]
位置编码: [0.84, 0.54, 0.00]
相加结果: [1.271471, 0.690302, 0.89005]
```

```T1~T6```: 注意力权重矩阵的某一行,表示当前 token 对句子中 每个 token 的关注度。

例子（第 1 个 token 的 attention 分布）：

```md
T1=0.31581  (自己: 31.6%)
T2=0.239435 (关注 token2: 23.9%)
...
T6=0.229044 (关注 token6: 22.9%)
```

```o1~o3```: Self-Attention 的最终输出向量, 即每个 token 的上下文感知表示。



回到教材：图 3.8 展示了如何计算查询 token 与每个输入 token 之间的中间注意力得分。我们通过计算查询 x(2) 与每个其他输入 token 的点积来确定这些得分（这里只是简单的将第二个token作为query vector进行了q与k的dot product）：

In [5]:
query = inputs[1]                                               #A
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)
print(attn_scores_2)


#A 第二个输入 token 用作查询向量

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [6]:
# 使用 softmax 函数来进行归一化。
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


> [!NOTE]
> 1. Softmax 的好处
> 
> - ```归一化输出为概率```：Softmax 将输出转换为 0 到 1 之间的概率，且所有类别的概率之和为 1，方便解释结果。例如，在分类任务中，输出可以直接表示模型对各类别的信心。
> 
> - ```平滑和放大效果```：Softmax 不仅能归一化，还具有平滑和放大效果。较大的输入值会被放大，较小的输入值会被抑制，从而增强模型对最优类别的区分。
>
> - ```支持多分类问题```：与 sigmoid 不同，Softmax 适用于多类别分类问题。它可以输出每个类别的概率，使得模型可以处理多分类任务。
>
> 2. 神经网络为什么喜欢使用 Softmax
>
> ```在神经网络中```，特别是分类模型（如图像分类、文本分类）中，Softmax 层通常用作最后一层输出。原因包括：
> 
> - ```便于优化```：在分类任务中，Softmax 输出的概率分布可与真实的标签概率进行比较，从而计算交叉熵损失。交叉熵损失的梯度较为稳定，便于模型的优化。
>
> - ```概率解释```：Softmax 输出可以解释为“模型对每个类别的信心”，使得输出直观可理解。
>
> - ```与交叉熵的结合```：Softmax 与交叉熵损失函数结合效果特别好，可以直接将模型预测的概率分布与真实标签比较，从而更快收敛，效果更好。

In [7]:
#计算最后的结果，即上下文向量
query = inputs[1] # 2nd input token is the query
context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


![Alt text](imgs/PixPin_2025-07-08_14-28-30.png)

![Alt text](imgs/PixPin_2025-07-08_14-29-10.png)

In [8]:
attn_scores = torch.empty(6, 6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [9]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [10]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


## 实现带有可训练权重的自注意力机制

![Alt text](imgs/PixPin_2025-07-08_14-40-53.png)

这里的权重指的是W_q, W_k, W_v

In [11]:
x_2 = inputs[1]                                                   #A
d_in = inputs.shape[1]                                            #B
d_out = 2                                                         #C


#A 第二个输入元素
#B 输入维度， d_in=3
#C 输出维度， d_out=2
#请注意，在 GPT 类模型中，输入维度和输出维度通常是相同的。
#不过，为了便于说明和更清楚地展示计算过程，我们在此选择了不同的输入（d_in=3）和输出（d_out=2）维度。

In [12]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)

In [15]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(f"query_2 is {query_2}, \nkey_2 is {key_2}, \nvalue_2 is {value_2}")

query_2 is tensor([0.4306, 1.4551], grad_fn=<SqueezeBackward4>), 
key_2 is tensor([0.4433, 1.1419], grad_fn=<SqueezeBackward4>), 
value_2 is tensor([0.3951, 1.0037], grad_fn=<SqueezeBackward4>)


> [!NOTE]
> 权重参数 VS 注意力权重
> 
> 请注意，在权重矩阵 W 中，术语“权重”是“权重参数”的缩写，指的是神经网络在训练过程中被优化的数值参数。这与注意力权重不同，注意力权重用于确定上下文向量对输入文本的不同部分的依赖程度，即神经网络对输入不同部分的关注程度。
> 
> 总之，权重参数是神经网络的基本学习系数，用于定义网络层之间的连接关系，而注意力权重则是根据上下文动态生成的特定值，用于衡量不同词语或位置在当前上下文中的重要性。

In [16]:
keys = inputs @ W_key
values = inputs @ W_value
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


In [17]:
keys_2 = keys[1]                                                  #A
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

#A 请牢记在Python中索引从0开始

tensor(1.8524, grad_fn=<DotBackward0>)


In [18]:
attn_scores_2 = query_2 @ keys.T # All attention scores for given query
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
       grad_fn=<SqueezeBackward4>)


In [19]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
       grad_fn=<SoftmaxBackward0>)


![Alt text](imgs/PixPin_2025-07-08_14-59-40.png)

In [20]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210], grad_fn=<SqueezeBackward4>)


![Alt text](imgs/PixPin_2025-07-08_15-29-33.png)

在这段 PyTorch 代码中，```SelfAttention_v1``` 是一个从 ```nn.Module``` 派生的类。```nn.Module``` 是 PyTorch 模型的基础组件，提供了创建和管理模型层所需的必要功能。

```__init__``` 方法初始化了用于计算查询（query）、键（key）和值（value）的可训练权重矩阵（```W_query```、```W_key``` 和 ```W_value```），每个矩阵都将输入维度 ```d_in``` 转换为输出维度 ```d_out```。

前向传播过程在 ```forward``` 方法中实现，我们通过将查询（query）和键（key）相乘来计算注意力得分（attn_scores），并使用 softmax 对这些得分进行归一化。最后，我们使用这些归一化的注意力得分对值（value）加权，生成上下文向量。

In [28]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):
    ###
    ### d_in is the input dimension, d_out is the content_vector dimension
    ###
    def __init__(self, d_in, d_out):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        attention_scores = queries @ keys.T
        attention_weights = torch.softmax(attention_scores / keys.shape[-1]**0.5, dim = -1)
        content_vec = attention_weights @ values
        return content_vec
    
torch.manual_seed(123)
self_attention_v1 = SelfAttention_v1(d_in=d_in,d_out=d_out)
print(self_attention_v1(inputs))


tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


![Alt text](imgs/PixPin_2025-07-08_15-41-12.png)

In [26]:
#  Listing 3.2 A self-attention class using PyTorch's Linear layers
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        print(f" {self.W_key}\n")

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec
    
torch.manual_seed(123)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

 Linear(in_features=3, out_features=2, bias=False)

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)


练习 3.1：比较SelfAttention_v1和 SelfAttention_v2

请注意，SelfAttention_v2 中的 nn.Linear 层使用了一种不同的权重初始化方式，而 SelfAttention_v1 则使用 nn.Parameter(torch.rand(d_in, d_out)) 进行初始化。这导致两种机制生成的结果有所不同。为了验证 SelfAttention_v1 和 SelfAttention_v2 的其他部分是否相似，我们可以将 SelfAttention_v2 对象中的权重矩阵转移到 SelfAttention_v1 中，从而使两者生成相同的结果。

你的任务是将 SelfAttention_v2 实例中的权重正确分配给 SelfAttention_v1 实例。为此，你需要理解两个版本中权重之间的关系。（提示：nn.Linear 存储的是转置形式的权重矩阵。）分配完成后，你应该能观察到两个实例生成相同的输出。

In [31]:
self_attention_v1.W_key.data = sa_v2.W_key.weight.T
self_attention_v1.W_query.data = sa_v2.W_query.weight.T
self_attention_v1.W_value.data = sa_v2.W_value.weight.T

out_v1 = self_attention_v1(inputs)
out_v2 = sa_v2(inputs)

print("Difference:", torch.abs(out_v1 - out_v2).max())

Difference: tensor(0., grad_fn=<MaxBackward1>)


## 3.5 使用因果注意力机制来屏蔽后续词

因果注意力（也称为掩蔽注意力,mask）是一种特殊的自注意力形式。它限制模型在处理任何给定的 token 时，只能考虑序列中的前一个和当前输入，而不能看到后续的内容。这与标准的自注意力机制形成对比，后者允许模型同时访问整个输入序列。

![Alt text](imgs/PixPin_2025-07-08_16-19-06.png)

In [35]:
queries = sa_v2.W_query(inputs)                                   #A
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)
print(f"attn_weights are: \n{attn_weights}\n")

#A 为了方便起见，我们复用上一节中 SelfAttention_v2 对象的query和key权重矩阵。

context_length = attn_scores.shape[0]
print(f"context_length is {context_length}\n")
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(f"mask_simple is \n{mask_simple}\n")
masked_simple = attn_weights*mask_simple
print(f"hence the final masked_simple is \n{masked_simple}")

attn_weights are: 
tensor([[0.1717, 0.1762, 0.1761, 0.1555, 0.1627, 0.1579],
        [0.1636, 0.1749, 0.1746, 0.1612, 0.1605, 0.1652],
        [0.1637, 0.1749, 0.1746, 0.1611, 0.1606, 0.1651],
        [0.1636, 0.1704, 0.1702, 0.1652, 0.1632, 0.1674],
        [0.1667, 0.1722, 0.1721, 0.1618, 0.1633, 0.1639],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<SoftmaxBackward0>)

context_length is 6

mask_simple is 
tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])

hence the final masked_simple is 
tensor([[0.1717, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1636, 0.1749, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1637, 0.1749, 0.1746, 0.0000, 0.0000, 0.0000],
        [0.1636, 0.1704, 0.1702, 0.1652, 0.0000, 0.0000],
        [0.1667, 0.1722, 0.1721, 0.1618, 0.1633, 0.0000],
        [0.1624, 0.1709, 0.17

In [36]:
# and we want to softmax the masked_simple

row_sums = masked_simple.sum(dim=1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3190, 0.3408, 0.3402, 0.0000, 0.0000, 0.0000],
        [0.2445, 0.2545, 0.2542, 0.2468, 0.0000, 0.0000],
        [0.1994, 0.2060, 0.2058, 0.1935, 0.1953, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<DivBackward0>)


归一化后我们可以看到，被掩盖的部分并不会影响当前的token，未被掩盖的概率和始终为1

Softmax 函数将输入值转换为概率分布。当一行中存在负无穷值（-∞）时，Softmax 函数会将这些值视为零概率。（从数学上讲，这是因为 e−∞ 接近于 0。）

我们可以通过创建一个对角线以上全为 1 的掩码，然后将这些 1 替换为负无穷大（-inf）值，从而实现这种更高效的掩码技巧：

In [39]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(f"masked: \n{masked}")
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5,dim=1)
print(f"attn_weights: \n{attn_weights}")

masked: 
tensor([[0.3111,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.1655, 0.2602,   -inf,   -inf,   -inf,   -inf],
        [0.1667, 0.2602, 0.2577,   -inf,   -inf,   -inf],
        [0.0510, 0.1080, 0.1064, 0.0643,   -inf,   -inf],
        [0.1415, 0.1875, 0.1863, 0.0987, 0.1121,   -inf],
        [0.0476, 0.1192, 0.1171, 0.0731, 0.0477, 0.0966]],
       grad_fn=<MaskedFillBackward0>)
attn_weights: 
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3190, 0.3408, 0.3402, 0.0000, 0.0000, 0.0000],
        [0.2445, 0.2545, 0.2542, 0.2468, 0.0000, 0.0000],
        [0.1994, 0.2060, 0.2058, 0.1935, 0.1953, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<SoftmaxBackward0>)


```Dropout``` 在深度学习中是一种技术，即在训练过程中随机忽略一些隐藏层单元，实际上将它们“丢弃”。这种方法有助于防止过拟合，确保模型不会过于依赖任何特定的隐藏层单元组合。需要特别强调的是，Dropout 仅在```训练过程中使用```，训练```结束后则会禁用```。

![Alt text](imgs/PixPin_2025-07-08_16-44-01.png)

在以下代码示例中，我们使用了50%的 dropout 率，这意味着屏蔽掉一半的注意力权重。（在后续章节中训练 GPT 模型时，我们将使用更低的 dropout 率，比如 0.1 或 0.2）

In [40]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6,6)
print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


**1. 为什么要进行缩放？**

* **保持训练／推理时激活量一致**
  在训练时，Dropout 会以概率 $p$ 把一部分神经元输出置为 0，剩下的神经元按比例放大，否则整体的信号强度就会变小，导致推理（eval）时和训练时分布不一致。
* **数学上：保持期望不变**
  假设原始激活值是 $x$，Dropout 丢弃概率是 $p$：

  * “保留”时，我们把该神经元的输出设为 $\displaystyle \frac{x}{1-p}$
  * “丢弃”时，输出设为 0
    这样，训练时单个神经元的**期望输出**

  $$
    E[\text{output}] = (1-p)\times \frac{x}{1-p} + p\times 0 = x,
  $$

  和原始 $x$ 一致。
* **推理时**（`model.eval()`），Dropout 不丢弃也不缩放，直接输出原始 $x$。

---

**2. Dropout 如何影响过拟合？**

1. **抑制“协同适应”（Co-adaptation）**
   如果所有神经元都始终在线，它们有可能“互相依赖”——某些特征检测器只在别的检测器存在时才有用。Dropout 随机屏蔽，让每个神经元都必须独立“扛起”解决问题的责任，更加鲁棒。
2. **相当于训练了一个“子网络”集合**
   每次前向都随机抽掉部分节点，就像在训练很多不同的小网络（同一个大网络的不同子集）。最终学到的权重，相当于这些子网络输出的平均，也类似 **Bagging**，能显著降低方差、减轻过拟合。
3. **增加噪声，提高泛化**
   随机屏蔽相当于在隐藏层注入噪声，模型必须学会在噪声中提取可靠信号，就不会在训练数据上“死记硬背”噪声／特定模式，从而提高对新样本的泛化能力。

---

 🔑 小结

* **缩放**：保证训练／推理阶段激活分布一致，不用在推理时再额外调整权重。
* **减轻过拟合**：让网络更具鲁棒性、降低权重方差、抑制特征间过度协同，从而在新数据上表现更好。


In [41]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0335, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6804, 0.0000, 0.0000, 0.0000],
        [0.4889, 0.5090, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3988, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3418, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


实现一个简洁的因果注意力类

In [42]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)                                              #A

#A 2个输入，每个输入有6个token，每个token的嵌入维度为3

torch.Size([2, 6, 3])


In [45]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length,context_length),diagonal=1))          #B
    
    def forward(self, x):
        b, num_tokens, d_in = x.shape                               #C
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.transpose(1,2)
        # D. 原地把“未来”位置的分数置为 -inf（softmax 后权重≈0），保证 causal。
        #    masked_fill_ 会直接在 attn_scores 的内存上修改，避免多余拷贝。
        attn_scores.masked_fill_(                                 #D
        self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context_vec = attn_weights @ values
        return context_vec
    
#B register_buffer 调用也是新添加的内容（后续内容会提供更多相关信息）
#C 我们交换第 1 和第 2 个维度，同时保持批次维度在第1个位置（索引0）
#D 在 PyTorch 中，带有下划线后缀的操作会在原有内存空间执行，直接修改变量本身，从而避免不必要的内存拷贝

虽然新增的代码行与之前章节介绍的内容基本一致，但我们现在在 __init__ 方法中添加了 self.register_buffer() 的调用。register_buffer 在 PyTorch 中并非所有情况下都必须使用，但在这里有其独特的优势。例如，当我们在大语言模型（LLM）中使用 CausalAttention 类时，buffer 会自动随模型迁移到合适的设备（CPU 或 GPU）。这意味着我们无需手动确保这些张量与模型参数在同一设备上，从而避免设备不匹配错误。

keys.transpose(1, 2)的作用：keys 的形状是 (batch_size, seq_len, d_out)，而我们要做的是把每个位置的 query 向量和所有位置的 key 向量做点积，也就是把 key 矩阵“转置”成 (batch_size, d_out, seq_len)，才能继续计算

In [46]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print("context_vecs.shape:", context_vecs.shape)

context_vecs.shape: torch.Size([2, 6, 2])


## 3.6 从单头注意力扩展到多头注意力

在实际应用中，实现多头注意力需要创建多个自注意力机制的实例（在 3.4.1 节的图 3.18 中已有展示），每个实例都具有独立的权重，然后将它们的输出合并。多个自注意力机制实例的应用属于计算密集型（CPU密集型）操作，但它对于识别复杂模式至关重要，这是基于 Transformer 的大语言模型所擅长的能力之一。

![Alt text](imgs/PixPin_2025-07-08_17-07-15.png)

In [47]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias)
             for _ in range(num_heads)])
        
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


![Alt text](imgs/PixPin_2025-07-08_17-10-02.png)

In [51]:
torch.manual_seed(123)
context_length = batch.shape[1] # This is the number of tokens
print(context_length)
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

6
tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


由此生成的 context_vecs 张量的第一个维度是 2，因为我们有两个输入文本（输入文本被复制，因此它们的上下文向量完全相同）。第二个维度对应每个输入中的 6 个 token。第三个维度对应每个 token 的 4 维嵌入向量